In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
import numpy as np
%matplotlib inline

import seaborn as sns

import pandas as pd

import argparse
import ellc

import gatspy
from gatspy import datasets, periodic
from gatspy.periodic import LombScargleMultiband, LombScargle

import astropy
from astropy import units, constants

import csv
import pylab

from scipy import stats

In [ ]:
dat = pd.read_csv('Thesis_dat_42918.csv', sep=",", header=0)
#print (dat)

In [ ]:
PeriodIn = dat["PERIOD"]
PeriodOut = dat["LSM_PERIOD"]
INCL = dat["INCLINATION"]
MIN_INCL = dat["MIN_INCLINATION"]
mass_1 = dat["MASS_1"]
mass_2 = dat["MASS_2"]
mass_rat = mass_2/mass_1
radius_1 = dat["RADIUS_1"]
radius_2 = dat["RADIUS_2"]
radius_rat = radius_2/radius_1
AppMagError = dat["appmag_error"]
InclinationError = dat["inclination_error"]
PeriodError = dat["period_error"]
RadiusError = dat["radius_error"]
#will change to just radius_error for new data
MaxAppmag = dat["max(app_magnitude)"]
delta_mag = dat["delta_mag"]
Chi2 = dat["chi2"]
delmag = abs(MaxAppmag - delta_mag)
Avg_sigma = dat["mean(dmag)"]
dist_kpc = dat['dist_kpc']
ecc = dat['eccentricity']
Sigma_Period_Whole = abs(PeriodOut - PeriodIn)/PeriodIn
Sigma_Period_Half = abs(PeriodOut - 0.5*PeriodIn)/(0.5*PeriodIn)
Sigma_Period_Twice = abs(PeriodOut - 2*PeriodIn)/(2*PeriodIn)

obs = np.where(PeriodOut != -999)[0]

ten = np.where(PeriodIn <= 10.)[0]
tenten = np.where(PeriodOut[ten] != -999)[0]

#prsamag = np.where(np.logical_and(MaxAppmag <= 24., MaxAppmag >= 16.))[0]
#prsa = np.where(PeriodIn[prsamag] <= 1000.)[0]
prsa = np.where(np.logical_and( np.logical_and(MaxAppmag <= 22., MaxAppmag >= 16.), PeriodIn <=1000))[0]
prsaobs = np.where(PeriodOut[prsa] != -999)[0]

usetwice = np.where(np.logical_and(np.isfinite(Sigma_Period_Twice), Sigma_Period_Twice <= 0.1))[0]
usehalf = np.where(np.logical_and(np.isfinite(Sigma_Period_Half), Sigma_Period_Half <= 0.1))[0]
usewhole = np.where(np.logical_and(np.isfinite(Sigma_Period_Whole), Sigma_Period_Whole <= 0.1))[0]

usetwice10 = np.where(np.logical_and(np.isfinite(Sigma_Period_Twice[ten]), Sigma_Period_Twice[ten] <= 0.1))[0]
usehalf10 = np.where(np.logical_and(np.isfinite(Sigma_Period_Half[ten]), Sigma_Period_Half[ten] <= 0.1))[0]
usewhole10 = np.where(np.logical_and(np.isfinite(Sigma_Period_Whole[ten]), Sigma_Period_Whole[ten] <= 0.1))[0]

usetwicep = np.where(np.logical_and(np.isfinite(Sigma_Period_Twice[prsa]), Sigma_Period_Twice[prsa] <= 0.1))[0]
usehalfp = np.where(np.logical_and(np.isfinite(Sigma_Period_Half[prsa]), Sigma_Period_Half[prsa] <= 0.1))[0]
usewholep = np.where(np.logical_and(np.isfinite(Sigma_Period_Whole[prsa]), Sigma_Period_Whole[prsa] <= 0.1))[0]

use = np.concatenate((usehalf, usewhole, usetwice), axis=0)

use10 = np.concatenate((usehalf10, usewhole10, usetwice10), axis=0)

usep = np.concatenate((usehalfp, usewholep, usetwicep), axis=0)


n_i = len(np.where(InclinationError ==1)[0])
n_p = len(np.where(PeriodError ==1)[0])
n_r = len(np.where(RadiusError == 1)[0])
n_am = len(np.where(AppMagError ==1)[0])



N0 = len(PeriodOut)
N1 = len(np.where(PeriodOut != -999)[0])
N2 = float(len(use))
N10 = float(len(tenten))
N11 = float(len(use10))

Npobs = float(len(prsaobs))
Npid = float(len(usep))

print ("For periods under 10 days - identification = ", N11/N10 * 100, "%")
print ("For Prsa conditions - periods 0-1000, appmag 16-22 - identification = ", Npid/Npobs * 100, "%")

print ("total number of binaries RUN = ", len(PeriodOut))
print ("total number of binaries NOT EXCLUDED (OBSERVED) = ", len(np.where(PeriodOut != -999)[0]))
print ("##################################")
print ("number binaries excluded for INCLINATION errors = ", len(np.where(InclinationError == 1)[0]))
print ("PERCENT INCLINATION errors = ", (n_i/(N0- N1))* 100, "%")
print ("number binaries excluded for PERIOD errors = ", len(np.where(PeriodError ==1)[0]))
print ("PERCENT PERIOD errors = ", (n_p/(N0- N1))* 100, "%")
print ("number of binaries excluded for APPARENT MAGNITUDE errors = ", len(np.where(AppMagError ==1)[0]))
print ("PERCENT APPARENT MAGNITUDE errors = ", (n_am/(N0- N1))* 100, "%")
print ("number binaries excluded for RADIUS errors = ", len(np.where(RadiusError == 1)[0]))
print ("PERCENT RADIUS errors = ", (n_r/(N0- N1))* 100, "%")
print ("##################################")
print ("total number of acceptable ~IDENTIFICATIONS~ = ", len(use))
print ("number WHOLE periods returned = ", len(usewhole))
print ("number HALF periods returned = ", len(usehalf))
print ("number TWICE periods returned = ", len(usetwice))
print ("##################################")
print ("Percent of run binaries observed = ", (N1/N0)*100, "%")
print("Percent of observed binaries identified = ",(N2/N1) * 100, "%")

In [ ]:
mpl.rcParams.update({'font.size': 40})
mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['ytick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 3
mpl.rcParams['ytick.minor.width'] = 3
mpl.rcParams['xtick.major.size'] = 15
mpl.rcParams['ytick.major.size'] = 15
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.width'] = 3

# Period #

In [ ]:
pcd = plt.figure(figsize=(30,15))
values, base = np.histogram(PeriodIn, bins=np.sort(PeriodIn))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
#make an additional cumulative dist for PeriodIn < 5d, or 10d?
plt.step(base[:-1], cumulative, linestyle = ':', linewidth = 5, color = 'k', label = "All binaries")
values, base = np.histogram(PeriodIn[obs], bins=np.sort(PeriodIn[obs]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-.', linewidth = 5, color='k', label = "LSST-detectable EBs")
values, base = np.histogram(PeriodIn[use], bins=np.sort(PeriodIn[use]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-', linewidth = 5, color='k', label = "Identified EBs")
plt.xlabel("Input period (days) ")
plt.ylabel("Fraction of population")
plt.legend(loc = 'best')
plt.xlim(0,20)
#Totally arbitrary limit... maybe would work better in log?
plt.show()
pcd.set_rasterized(True)
pcd.savefig("periodin_cumulativedistribution.eps")
pcd.savefig("periodin_cumulativedistribution.pdf")

logpcd = plt.figure(figsize=(30,15))
values, base = np.histogram(np.log10(PeriodIn), bins=np.sort(np.log10(PeriodIn)))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
#make an additional cumulative dist for PeriodIn < 5d, or 10d?
plt.step(base[:-1], cumulative, linestyle = ':', linewidth = 5, color = 'k', label = "All binaries")
values, base = np.histogram(np.log10(PeriodIn[obs]), bins=np.sort(np.log10(PeriodIn[obs])))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-.', linewidth = 5, color='k', label = "LSST-detectable EBs")
values, base = np.histogram(np.log10(PeriodIn[use]), bins=np.sort(np.log10(PeriodIn[use])))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-', linewidth = 5, color='k', label = "Identified EBs")
plt.xlabel("log(Input period) (log(days)) ")
plt.ylabel("Fraction of poulation")
plt.legend(loc = 'best')
plt.xlim(0,7)
#Totally arbitrary limit... NOT better in log
plt.show()
logpcd.set_rasterized(True)
logpcd.savefig("logperiodin_cumulativedistribution.eps")
logpcd.savefig("logperiodin_cumulativedistribution.pdf")



# MASS #

In [ ]:
mcd = plt.figure(figsize=(20,15))
values, base = np.histogram(mass_rat, bins=np.sort(mass_rat))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
#make an additional cumulative dist for PeriodIn < 5d, or 10d?
plt.step(base[:-1], cumulative, linestyle = ':', linewidth = 5, color = 'k', label = "All binaries")
values, base = np.histogram(mass_rat[obs], bins=np.sort(mass_rat[obs]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-.', linewidth = 5, color='k', label = "LSST-detectable EBs")
values, base = np.histogram(mass_rat[use], bins=np.sort(mass_rat[use]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-', linewidth = 5, color='k', label = "Identified EBs")
plt.xlabel("Mass ratio ")
plt.ylabel("Fraction of population")
plt.legend(loc = 'best', prop={'size': 30})
plt.xlim(0,4)
#plt.xlim(0,11.5)
plt.show()
mcd.set_rasterized(True)
mcd.savefig("massrat_cumulativedistribution.eps")
mcd.savefig("massrat_cumulativedistribution.pdf")



# RADIUS #

In [ ]:
rcd = plt.figure(figsize=(20,15))
values, base = np.histogram(np.log10(radius_rat), bins=np.sort(np.log10(radius_rat)))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
#make an additional cumulative dist for PeriodIn < 5d, or 10d?
plt.step(base[:-1], cumulative, linestyle = ':', linewidth = 5, color = 'k', label = "All binaries")
values, base = np.histogram(np.log10(radius_rat[obs]), bins=np.sort(np.log10(radius_rat[obs])))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-.', linewidth = 5, color='k', label = "LSST-detectable EBs")
plt.xlabel("log(Radius ratio) ")
values, base = np.histogram(np.log10(radius_rat[use]), bins=np.sort(np.log10(radius_rat[use])))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-', linewidth = 5, color='k', label = "Identified EBs")
plt.xlabel("log(Radius ratio) ")
plt.ylabel("Fraction of population")
plt.legend(loc = 'best', prop={'size': 30})
#plt.xlim(0,11.5)
plt.show()
rcd.set_rasterized(True)
rcd.savefig("radiusrat_cumulativedistribution.eps")
rcd.savefig("radiusrat_cumulativedistribution.pdf")

# ECCENTRICITY #

In [ ]:
ecd = plt.figure(figsize=(20,15))
values, base = np.histogram(ecc, bins=np.sort(ecc))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
#make an additional cumulative dist for PeriodIn < 5d, or 10d?
plt.step(base[:-1], cumulative, linestyle = ':', linewidth = 5, color = 'k', label = "All binaries")
values, base = np.histogram(ecc[obs], bins=np.sort(ecc[obs]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-.', linewidth = 5, color='k', label = "LSST-detectable EBs")
values, base = np.histogram(ecc[use], bins=np.sort(ecc[use]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-', linewidth = 5, color='k', label = "Identified EBs")
plt.xlabel("Eccentricity ")
plt.ylabel("Fraction of population")
plt.legend(loc = 'best', prop={'size': 30})
#plt.xlim(0,11.5)
plt.show()
ecd.set_rasterized(True)
ecd.savefig("ecc_cumulativedistribution.eps")
ecd.savefig("ecc_cumulativedistribution.pdf")

# DISTANCE (kpc) #

In [ ]:
dcd = plt.figure(figsize=(20,15))
values, base = np.histogram(dist_kpc, bins=np.sort(dist_kpc))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
#make an additional cumulative dist for PeriodIn < 5d, or 10d?
plt.step(base[:-1], cumulative, linestyle = ':', linewidth = 5, color = 'k', label = "All binaries")
values, base = np.histogram(dist_kpc[obs], bins=np.sort(dist_kpc[obs]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-.', linewidth = 5, color='k', label = "LSST-detectable EBs")
values, base = np.histogram(dist_kpc[use], bins=np.sort(dist_kpc[use]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-', linewidth = 5, color='k', label = "Identified EBs")
plt.xlabel("Distance (kpc)")
plt.ylabel("Fraction of population")
plt.legend(loc = 'best', prop={'size': 30})
#plt.xlim(0,11.5)
plt.show()
dcd.set_rasterized(True)
dcd.savefig("distkpc_cumulativedistribution.eps")
dcd.savefig("distkpc_cumulativedistribution.pdf")

# INCLINATION #

In [ ]:
icd = plt.figure(figsize=(20,15))
values, base = np.histogram(INCL, bins=np.sort(INCL))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
#make an additional cumulative dist for PeriodIn < 5d, or 10d?
plt.step(base[:-1], cumulative, linestyle = ':', linewidth = 5, color = 'k', label = "All binaries")
values, base = np.histogram(INCL[obs], bins=np.sort(INCL[obs]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-.', linewidth = 5, color='k', label = "LSST-detectable EBs")
values, base = np.histogram(INCL[use], bins=np.sort(INCL[use]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-', linewidth = 5, color='k', label = "Identified EBs")
plt.xlabel("Inclination")
plt.ylabel("Fraction of population")
plt.legend(loc = 'best', prop={'size': 30})
plt.xlim(0,90)
plt.show()
icd.set_rasterized(True)
icd.savefig("incl_cumulativedistribution.eps")
icd.savefig("incl_cumulativedistribution.pdf")

# APPARENT MAGNITUDE #

In [ ]:
maxcd = plt.figure(figsize=(20,15))
values, base = np.histogram(MaxAppmag, bins=np.sort(MaxAppmag))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
#make an additional cumulative dist for PeriodIn < 5d, or 10d?
plt.step(base[:-1], cumulative, linestyle = ':', linewidth = 5, color = 'k', label = "All binaries")
values, base = np.histogram(MaxAppmag[obs], bins=np.sort(MaxAppmag[obs]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-.', linewidth = 5, color='k', label = "LSST-detectable EBs")
values, base = np.histogram(MaxAppmag[use], bins=np.sort(MaxAppmag[use]))
cumulative = np.cumsum(values)
cumulative = cumulative/max(cumulative)
plt.step(base[:-1], cumulative, linestyle = '-', linewidth = 5, color='k', label = "Identified EBs")
plt.xlabel("Maximum apparent magnitude")
plt.ylabel("Fraction of population")
plt.legend(loc = 'best', prop={'size': 30})
#plt.xlim(80,90)
plt.show()
maxcd.set_rasterized(True)
maxcd.savefig("maxappmag_cumulativedistribution.eps")
maxcd.savefig("maxappmag_cumulativedistribution.pdf")